In [3]:
import pandas as pd
import sqlite3


In [4]:
# Query para ver el informe del crimen
query1 = '''
SELECT *
FROM crime_scene_report
WHERE ("date" LIKE 20180115) AND (type = "murder") AND (city LIKE "SQL City");
'''
#Security footage shows that there were 2 witnesses. 
# The first witness lives at the last house on "Northwestern Dr". 
# The second witness, named Annabel, lives somewhere on "Franklin Ave".


# Query para saber quién es testigo_1
query2 = '''
SELECT a.name, a.address_street_name, a.address_number, b.transcript
FROM person AS a
LEFT JOIN interview AS b
ON a.id = b.person_id
WHERE (address_street_name LIKE "%Northwestern Dr%")
'''

# Morty Schapiro
#Northwestern Dr 4919
# I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. 
# The membership number on the bag started with "48Z".
# Only gold members have those bags. The man got into a car with a plate that included "H42W".

# Query del testigo_2

query3 = '''
SELECT a.name, a.address_street_name, a.address_number, b.transcript
FROM person AS a
LEFT JOIN interview AS b
ON a.id = b.person_id
WHERE (address_street_name LIKE "%Franklin Ave%") AND (a.name LIKE "%Annabel%")
'''

#transcript de Annabel Miller que vive en Franklin Ave 103
#I saw the murder happen, and I recognized the killer from my gym when 
# I was working out last week on January the 9th.


# Query filtrando por la pista que da Morty

query4 = '''
SELECT * 
FROM "get_fit_now_member"
WHERE (membership_status = "gold") AND (id LIKE "%48Z%")
'''

# Joe Germuska ID48Z7A
# Jeremy Bowers ID48Z55

# Query para saber quien tiene la matricula
query5 = '''
SELECT a.name, b.plate_number
FROM person AS a
LEFT JOIN drivers_license AS b
ON a.license_id = b.id
WHERE (a.id = 28819) OR (a.id = 67318)
AND b.plate_number LIKE "%H42W%"
'''
#JEREMY BOWERS tiene la maticula 0H42W2

# Query para ver el testimonio Jeremy
query6 = '''
SELECT a.name, b.transcript
FROM person AS a
LEFT JOIN interview AS b
ON a.id = b.person_id
WHERE a.id = 67318
'''

#Testimonio de Jeremy
#I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") 
# or 5'7" (67"). She has red hair and she drives a Tesla Model S. 
# I know that she attended the SQL Symphony Concert 3 times in December 2017.

# Query para buscar a la mujer que contrató al asesino
# Buscamos por sus características.
query7 = ''' 
SELECT *
FROM drivers_license
WHERE hair_color = "red"
AND gender = "female"
AND (height > 65 AND height < 67)
AND hair_color = "red"
AND car_model LIKE "%Model%"
'''
# matricula 500123 edad 68 y ID 202298
# matricula 08CM64 edad 65 y ID 291182


# Filtramos por ID para buscar su nombre
query8 = '''
SELECT a.id, a.name
FROM person AS a
LEFT JOIN drivers_license AS b
ON a.license_id = b.id
WHERE license_id = 202298 OR license_id = 291182
'''
#Regina George ID 90700
#Miranda Priestly ID 99716

#Filtramos por nombre el evento SQL Symphony Concert
query9 = '''
SELECT a.*, b.name
FROM facebook_event_checkin AS a
LEFT JOIN person AS b
ON a.person_id = b.id
WHERE a.person_id = 90700 OR a.person_id = 99716
'''

# vemos una tabla donde indica que la asesina es Miranda Priestly y que acudió tres veces a este evento

In [7]:
def welcome():
    print('''Welcome Mr. Gadget!
          A crime has taken place and the detective needs your help. 
          The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **murder** that occurred sometime on **Jan.15, 2018** and that it took place in **SQL City**. 
          Start by retrieving the corresponding crime scene report from the police department’s database.''')

def clue_1():
    print('''we start by rescuing the tracks that we remember:
          - Crime: murder
          - Date: 15 of January, 2018
          - City: SQL City
''')
    
connection = sqlite3.connect("/Users/patri/Desktop/DS24/ONLINE_DS_THEBRIDGE_PGF/SPRINT_6/TEAM_CHALLENGE/sql-murder-mystery.db")


cursor = connection.cursor()

#Report
def report():
    report = pd.read_sql(query1, connection)
    print("crime scene report:")
    for line in report["description"]:
        print(line)

def clue_2():
    print('''\nMore clues:\n
          - 1st witness: lives at the last house on "Northwestern Dr
          - 2nd witness: named Annabel, lives somewhere on "Franklin Ave
''')

#First Testimony
def witness_1():
    df_testimony_1 = pd.read_sql(query2, connection)
    number = df_testimony_1["address_number"].max() #last house
    df_witness_1_num = df_testimony_1[df_testimony_1["address_number"] == number]
    df_witness_1 = df_witness_1_num[["name", "transcript", "address_number"]]
    testimony = df_witness_1["transcript"]
    for line in testimony:
        testimony_1 = line
        print("Testimony of the first witness:\n")
        print(f"\t{testimony_1.replace(".", "\n")}")



# Testimonio del testigo 2

def witness_2():
    df_witness_2 = pd.read_sql(query3, connection)
    testimony_2 = df_witness_2["transcript"]
    for line in testimony_2:
        testimony_num_2 = line
        print("\nTestimony of the second witness:\n")
        print(f"\t{testimony_num_2.replace(",", "\n")}")

def clue_3():
       print('''The cluses we found:
          - He had a "Get Fit Now Gym" bag. 
          - The membership number on the bag started with "48Z".
          - Only gold members have those bags. The man got into a car with a plate that included "H42W"
          - He was at the gym on 9th January
''')

# Posibles asesinos
def searching_killer():
    killer = pd.read_sql(query4, connection)
    print("Members who are GOLD and which membership number starts with '48Z' are:")
    print(killer["name"][0])
    print(killer["name"][1])

def killers():
    killer = pd.read_sql(query5, connection)
    print("Investigating the license plate, we discover that the alleged murderer is:")
    print(f"{killer["name"][1]}")

def verification():
    print('''
          "Congrats, you found the murderer! 
          But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. 
          If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. 
          Use this same INSERT statement with your new suspect to check your answer."
    ''')

def clue_4():
    print('''
    we have to keep looking for our killer
    ''')

def testimony_of_killer():
    killer_testimony = pd.read_sql(query6, connection)
    killer_interview = killer_testimony["transcript"]
    for line in killer_interview:
        interview = line
        print("interview")

def clue_5():
    print('''
More clues:
          - a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67").
          - She has red hair and she drives a Tesla Model S. 
          - she attended the SQL Symphony Concert 3 times in December 2017.
''')

def killer_woman():
    killer = pd.read_sql(query9, connection)
    killer_name = killer["name"][0]
    print("The killer is:")
    print(killer_name)

def final_verification():
    print('''
Congrats, you found the brains behind the murder! 
Everyone in SQL City hails you as the greatest SQL detective of all time. 
Time to break out the champagne!
''')

In [8]:
welcome()
clue_1()
report()
clue_2()
witness_1()
witness_2()
clue_3()
searching_killer()
killers()
verification()
clue_4()
testimony_of_killer()
clue_5()
killer_woman()
final_verification()

Welcome Mr. Gadget!
          A crime has taken place and the detective needs your help. 
          The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **murder** that occurred sometime on **Jan.15, 2018** and that it took place in **SQL City**. 
          Start by retrieving the corresponding crime scene report from the police department’s database.
we start by rescuing the tracks that we remember:
          - Crime: murder
          - Date: 15 of January, 2018
          - City: SQL City

crime scene report:
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

More clues:

          - 1st witness: lives at the last house on "Northwestern Dr
          - 2nd witness: named Annabel, lives somewhere on "Franklin Ave

Testimony of the first witness:

	I heard a gunshot and then saw a man run out
 